<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/AllCF_Amp_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import  drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
nltk.download('all')
from nltk.corpus import sentiwordnet as swn
device_name = tf.test.gpu_device_name()
print(device_name)
df = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/InfluenceAnalysis/crowdFlower/cF_clean_our_combined.csv')
df = df.sample(frac=1,random_state=32)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

In [5]:
df.head()

,emotion,tweets
35975,1,Praying for love in a lap dance and paying in...
17590,2,trying to fix my internet connection gues...
38071,0,starting an account here on twitter
23304,2,hmmm wrong link ignore my tweet
26990,0,thanks before the major chop


In [6]:
df.emotion.unique()

array([1, 2, 0, 3])

In [7]:
no_labels=4
category_dict = {}
for i in range(no_labels):
  category_dict[i]=[0 for j in range(no_labels)]
  category_dict[i][i]=1
data_tweet = [str(x).lower().split() for x in df['tweets']]
data_cat = np.array([np.array(category_dict[x]) for x in df['emotion']])
print(data_cat[:5])

[[0 1 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 0]]


In [8]:
print(data_tweet[:5])
print(data_cat[:5])

[['praying', 'for', 'love', 'in', 'a', 'lap', 'dance', 'and', 'paying', 'in', 'naivety', 'less', 'than', 'one', 'of', 'my', 'fav', 'songs'], ['trying', 'to', 'fix', 'my', 'internet', 'connection', 'guess', 'my', 'prayers', 'have', 'been', 'answered', 'and', 'i', 'wont', 'have', 'any', 'study', 'distractions', 'ugh'], ['starting', 'an', 'account', 'here', 'on', 'twitter'], ['hmmm', 'wrong', 'link', 'ignore', 'my', 'tweet'], ['thanks', 'before', 'the', 'major', 'chop']]
[[0 1 0 0]
 [0 0 1 0]
 [1 0 0 0]
 [0 0 1 0]
 [1 0 0 0]]


In [9]:
# Parameters
Min_count = 0
Embedding_size = 200
Window_size = 5
Negative_sampling = 00

In [10]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)
w2v_sg.wv.init_sims(True)
w2v_cbow.wv.init_sims(True)

In [11]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/InfluenceAnalysis/glove/glove.twitter.27B.200d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [12]:
batch_size = 500
Max_input_size = max([len(x) for x in data_tweet])
data_tweet = [nltk.pos_tag(x) for x in data_tweet]
def getsent(word,tag):
  res=0
  try:
    x = swn.senti_synset(word+'.'+tag[0].lower()+'.01')
    res =  (x.pos_score()-x.neg_score())
  finally:
    return res 
data_tweet = [[(i[0],i[1],getsent(i[0],i[1])) for i in x] for x in data_tweet]

In [13]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor=2):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(w2v.wv.word_vec(i[0]),i[1]) for i in x if i[0] in vocab]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ' or tweet[i][1][:2]=='RB':
            tweet[i]=((tweet[i][0])*mul_factor,tweet[i][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = np.array(data_cat[k*(batch_size):(k+1)*(batch_size)])
      yield (temp,tempres)          
      
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor=2):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ' or tweet[i][1][:2]=='RB':
            tweet[i]=((tweet[i][0])*mul_factor,tweet[i][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = np.array(data_cat[k*(batch_size):(k+1)*(batch_size)])
      yield (temp,tempres)

In [14]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
sss.get_n_splits(data_tweet, data_cat)
X_train=[]
X_test=[]
y_train=[]
y_test=[]
counter=0
for train_index, test_index in sss.split(data_tweet, data_cat):
  print("TRAIN:", train_index, "TEST:", test_index)
  for i in train_index:
    X_train+=[data_tweet[i]]
    y_train+=[data_cat[i]]

  for j in test_index:
    X_test+=[data_tweet[j]]
    y_test+=[data_cat[j]]

 # X_train, X_test = data_tweet[train_index], data_tweet[test_index]
 # y_train, y_test = data_cat[train_index], data_cat[test_index]
print(X_train[:5])
print(y_train[:5])
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
sss.get_n_splits(X_train, y_train)
x_train=[]
X_val=[]
Y_train=[]
y_val=[]
for train_index1, test_index1 in sss.split(X_train, y_train):
  print("TRAIN:", train_index1, "TEST:", test_index1)
  for i in train_index1:
    x_train+=[X_train[i]]
    Y_train+=[y_train[i]]

  for j in test_index1:
    X_val+=[X_train[j]]
    y_val+=[y_train[j]]
X_train=x_train
y_train=Y_train
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))
print(len(X_val))
print(len(y_val))
print(X_train[:5])
print(y_train[:5])
result_table = [0,0,0]

TRAIN: [10470  6321 13813 ... 24250 39308  6334] TEST: [22946 37404 11668 ...  6796  7895 14585]
[[('the', 'DT', 0), ('goonies', 'NNS', 0), ('projected', 'VBN', 0), ('on', 'IN', 0), ('a', 'DT', 0), ('garage', 'NN', 0.0), ('door', 'NN', 0.0), ('with', 'IN', 0), ('my', 'PRP$', 0), ('friends', 'NNS', 0), ('amazing', 'VBG', 0)], [('17', 'CD', 0), ('again', 'RB', 0.0), ('tonight', 'NN', 0.0)], [('we', 'PRP', 0), ('finally', 'RB', 0.0), ('hit', 'VBD', 0.0), ('1000', 'CD', 0), ('friends', 'NNS', 0), ('thanks', 'NNS', 0.125)], [('lolzz', 'JJ', 0), ('public', 'JJ', 0), ('script', 'NN', 0.0), ('wait', 'NN', 0.0), ('n', 'NN', 0.0), ('watch', 'NN', 0.0)], [('night', 'NN', 0.0)]]
[array([0, 1, 0, 0]), array([1, 0, 0, 0]), array([0, 1, 0, 0]), array([0, 1, 0, 0]), array([1, 0, 0, 0])]
TRAIN: [28318 14444 15946 ... 11043 21820 15291] TEST: [23742   966  6886 ... 25081 24593 26654]
25600
25600
8000
8000
6400
6400
[[('geez', 'JJ', 0), ('15', 'CD', 0), ('years', 'NNS', 0.0), ('you', 'PRP', 0), ('d', 'VB

In [15]:
epochs = 30

In [16]:
from keras import backend as K
def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recalls = true_positives / (possible_positives + K.epsilon())
        return recalls
def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precisions = true_positives / (predicted_positives + K.epsilon())
        return precisions
def f1(y_true, y_pred):
    precisions = precision(y_true, y_pred)
    recalls = recall(y_true, y_pred)
    return 2*((precisions*recalls)/(precisions+recalls+K.epsilon()))

In [22]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dropout(0.2))
model_sg_bi.add(tf.keras.layers.Dense(64,activation='relu'))
model_sg_bi.add(tf.keras.layers.Dropout(0.2))
model_sg_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
sg_train_gen=vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size)
sg_val_gen=vec_gen(w2v_sg,vocab_sg,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_sg_bi = model_sg_bi.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)
result_table[2]=history[1]
print("SG_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
52/52 [==============================] - 62s 1s/step - loss: 1.2964 - accuracy: 0.3470 - f1: 0.0109 - precision: 0.0996 - recall: 0.0065 - val_loss: 1.1806 - val_accuracy: 0.4188 - val_f1: 0.0122 - val_precision: 0.8013 - val_recall: 0.0062
Epoch 2/30
52/52 [==============================] - 58s 1s/step - loss: 1.1741 - accuracy: 0.4230 - f1: 0.0838 - precision: 0.6177 - recall: 0.0469 - val_loss: 1.1479 - val_accuracy: 0.4592 - val_f1: 0.2239 - val_precision: 0.5915 - val_recall: 0.1383
Epoch 3/30
52/52 [==============================] - 59s 1s/step - loss: 1.1321 - accuracy: 0.4668 - f1: 0.2436 - precision: 0.6228 - recall: 0.1520 - val_loss: 1.1335 - val_accuracy: 0.4715 - val_f1: 0.2959 - val_precision: 0.5820 - val_recall: 0.1988
Epoch 4/30
52/52 [==============================] - 59s 1s/step - loss: 1.1085 - accuracy: 0.4873 - f1: 0.3037 - precision: 0.6259 - recall: 0.2009 - val_loss: 1.1167 - val_accuracy: 0.4825 - val_f1: 0.3151 - val_precision: 0.5894 - val_recall:

In [21]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dropout(0.2))
model_cbow_bi.add(tf.keras.layers.Dense(64,activation='relu'))
model_cbow_bi.add(tf.keras.layers.Dropout(0.2))
model_cbow_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
cbow_train_gen=vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size)
cbow_val_gen=vec_gen(w2v_cbow,vocab_cbow,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_cbow_bi = model_cbow_bi.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[1]=history[1]
print("CBOW_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
52/52 [==============================] - 64s 1s/step - loss: 1.2848 - accuracy: 0.3842 - f1: 0.0159 - precision: 0.2097 - recall: 0.0085 - val_loss: 1.1692 - val_accuracy: 0.4492 - val_f1: 0.0929 - val_precision: 0.6141 - val_recall: 0.0503
Epoch 2/30
52/52 [==============================] - 59s 1s/step - loss: 1.1640 - accuracy: 0.4433 - f1: 0.1954 - precision: 0.5918 - recall: 0.1185 - val_loss: 1.1454 - val_accuracy: 0.4554 - val_f1: 0.2422 - val_precision: 0.5829 - val_recall: 0.1531
Epoch 3/30
52/52 [==============================] - 59s 1s/step - loss: 1.1340 - accuracy: 0.4678 - f1: 0.2723 - precision: 0.6112 - recall: 0.1759 - val_loss: 1.1388 - val_accuracy: 0.4640 - val_f1: 0.3107 - val_precision: 0.5725 - val_recall: 0.2135
Epoch 4/30
52/52 [==============================] - 58s 1s/step - loss: 1.1124 - accuracy: 0.4868 - f1: 0.3140 - precision: 0.6293 - recall: 0.2098 - val_loss: 1.1216 - val_accuracy: 0.4757 - val_f1: 0.3306 - val_precision: 0.5882 - val_recall:

In [17]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dropout(0.2))
model_glove_bi.add(tf.keras.layers.Dense(64,activation='relu'))
model_glove_bi.add(tf.keras.layers.Dropout(0.2))
model_glove_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy',f1,precision,recall])
glove_train_gen=glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size)
glove_val_gen=glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_glove_bi = model_glove_bi.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[0]=history[1]
print("GLOVE_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
52/52 [==============================] - 34s 578ms/step - loss: 1.1991 - accuracy: 0.4197 - f1: 0.1311 - precision: 0.4772 - recall: 0.0823 - val_loss: 1.0661 - val_accuracy: 0.5349 - val_f1: 0.4277 - val_precision: 0.6031 - val_recall: 0.3315
Epoch 2/30
52/52 [==============================] - 29s 550ms/step - loss: 1.0457 - accuracy: 0.5540 - f1: 0.4534 - precision: 0.6394 - recall: 0.3519 - val_loss: 1.0342 - val_accuracy: 0.5531 - val_f1: 0.4756 - val_precision: 0.6301 - val_recall: 0.3822
Epoch 3/30
52/52 [==============================] - 29s 550ms/step - loss: 1.0112 - accuracy: 0.5670 - f1: 0.4928 - precision: 0.6584 - recall: 0.3943 - val_loss: 1.0174 - val_accuracy: 0.5585 - val_f1: 0.4951 - val_precision: 0.6271 - val_recall: 0.4092
Epoch 4/30
52/52 [==============================] - 29s 562ms/step - loss: 0.9819 - accuracy: 0.5855 - f1: 0.5188 - precision: 0.6677 - recall: 0.4246 - val_loss: 0.9951 - val_accuracy: 0.5692 - val_f1: 0.5067 - val_precision: 0.6465 -

IndexError: ignored

In [18]:
result_table[0]=history[1]
print("GLOVE_BI")

GLOVE_BI


In [23]:
#print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("glove",result_table[:1])
print("cbow",result_table[1:2])
print("sg",result_table[2:3])
# print(history_sg_bi.history)
# plt.title("Skip-Gram + Bi-LSTM accuracy, loss vs epochs Graph")
# plt.plot(history_sg_bi.history['loss'],c='b',label='loss')
# plt.plot(history_sg_bi.history['accuracy'],c='r',label='accuracy')
# plt.xlabel('epochs')
# plt.legend()
# # plt.yticks()
# plt.show()
# plt.title("GloVe + Bi-LSTM accuracy, loss vs epochs Graph")
# plt.plot(history_glove_bi.history['loss'],c='b',label='loss')
# plt.plot(history_glove_bi.history['accuracy'],c='r',label='accuracy')
# plt.xlabel('epochs')
# plt.legend()
# # plt.yticks()
# plt.show()

glove [0.5414999723434448]
cbow [0.512499988079071]
sg [0.5048750042915344]
